<a href="https://colab.research.google.com/github/willschuerman/lets-stop-wildfires-hackathon/blob/will/wildfire_smoke_challenge_1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
import tarfile
import pandas as pd
import os
from os import listdir
import numpy as np
import glob
from datetime import datetime
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

#2. Generate CSV with info about images

In [104]:
# Define datapath
camera_folders = listdir_nohidden('../data/train_data_set')

# Load file paths into dataframe
all_images = []
for camera_folder in camera_folders:
    image_fpaths = listdir_nohidden(camera_folder)
    all_images+=image_fpaths

d = {'fPath':all_images}
image_info = pd.DataFrame(d)
image_info['camera_name'] = image_info['fPath'].str.split('/',expand = True)[3]
image_info['fName'] = image_info['fPath'].str.split('/',expand = True)[4]
image_info['isSmoke'] = np.where(image_info['fName'].str.contains('\+'), 1, 0)

In [106]:
# mark image in which smoke first appears
first_smoke_images = ['1498416633_+00240.jpg',
'1499546383_+00120.jpg',
'1512674164_+00180.jpg',
'1528757466_+00060.jpg',
'1529020885_+00180.jpg',
'1529001020_+00060.jpg',
'1530739165_+00120.jpg',
'1532544985_+00060.jpg',
'1530901921_+00240.jpg',
'1559167445_+00060.jpg',
'1562959460_+00119.jpg',
'1563135083_+00060.jpg',
'1563122707_+00120.jpg',
'1563220703_-02100.jpg',
'1563228743_+01620.jpg',
'1564685261_+00060.jpg',
'1564869279_+00060.jpg',
'1565731242_+00000.jpg'] # easier than importing from a separate CSV file

# mark image in which smoke first appears
image_info['firstSmoke'] = np.where(image_info.fName.isin(first_smoke_images), 1, 0)

In [113]:
# add in time info
image_info['unixTime'] = image_info['fName'].str[0:10].astype(int)

image_info['dateTime'] = pd.to_datetime(image_info['unixTime'],unit='s')
image_info = image_info.sort_values(by=['camera_name','unixTime'])

# fix the index after sorting and concatenating
image_info.index = pd.RangeIndex(len(image_info.index)) 
image_info.head()

,fPath,camera_name,fName,isSmoke,firstSmoke,unixTime,dateTime
0,../data/train_data_set/20170625-BBM-bm-n-mobo/...,20170625-BBM-bm-n-mobo,1498413993_-02400.jpg,0,0,1498413993,2017-06-25 18:06:33
1,../data/train_data_set/20170625-BBM-bm-n-mobo/...,20170625-BBM-bm-n-mobo,1498414053_-02340.jpg,0,0,1498414053,2017-06-25 18:07:33
2,../data/train_data_set/20170625-BBM-bm-n-mobo/...,20170625-BBM-bm-n-mobo,1498414113_-02280.jpg,0,0,1498414113,2017-06-25 18:08:33
3,../data/train_data_set/20170625-BBM-bm-n-mobo/...,20170625-BBM-bm-n-mobo,1498414173_-02220.jpg,0,0,1498414173,2017-06-25 18:09:33
4,../data/train_data_set/20170625-BBM-bm-n-mobo/...,20170625-BBM-bm-n-mobo,1498414233_-02160.jpg,0,0,1498414233,2017-06-25 18:10:33


In [150]:
camera_names = list(image_info.camera_name.unique())
test_cameras = ["20190529-94Fire-lp-s-mobo-c", "20191102-bm-e-mobo-c-2 ","20180614-Bridle-hp-n-mobo-c"]
all_pairs = []
for camera in camera_names:
    print('Starting camera: {}'.format(camera))
    image_df = image_info[image_info['camera_name']==camera]
    image_df.sort_values(by='unixTime',inplace=True)
    image_df.reset_index(inplace=True)
    camera_pairs = []
    print('Finding camera pairs')
    label = 0
    for index in range(len(image_df)-1):
        if image_df.loc[index+1]['firstSmoke']:
            label = 1
        test_label = camera in test_cameras
        camera_pairs.append([image_df.loc[index]['fPath'],image_df.loc[index+1]['fPath'],label,test_label])
    all_pairs+=camera_pairs

Starting camera: 20170625-BBM-bm-n-mobo
Finding camera pairs
Starting camera: 20170708-Whittier-syp-n-mobo-c
Finding camera pairs
Starting camera: 20171207-Lilac-rm-s-mobo


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Finding camera pairs
Starting camera: 20180611-fallbrook-rm-w-mobo-c
Finding camera pairs
Starting camera: 20180614-Bridle-hp-n-mobo-c
Finding camera pairs
Starting camera: 20180614-Hope-wc-e-mobo-c
Finding camera pairs
Starting camera: 20180704-Benton-hp-n-mobo-c
Finding camera pairs
Starting camera: 20180706-West-lp-n-mobo-c
Finding camera pairs
Starting camera: 20180725-Cranston-hp-n-mobo-c
Finding camera pairs
Starting camera: 20190529-94Fire-lp-s-mobo-c
Finding camera pairs
Starting camera: 20190712-RockHouse-wc-e-mobo-c
Finding camera pairs
Starting camera: 20190714-MLOSouth-lo-s-mobo-c
Finding camera pairs
Starting camera: 20190714-PinosSouth-pi-s-mobo-c
Finding camera pairs
Starting camera: 20190715-MLOSouth2-lo-s-mobo-c
Finding camera pairs
Starting camera: 20190715-MLOSouth3-lo-s-mobo-c
Finding camera pairs
Starting camera: 20190801-Caliente-om-w-mobo
Finding camera pairs
Starting camera: 20190803-OtaySouth-om-s-mobo
Finding camera pairs
Starting camera: 20190813-Topanga-69br

In [151]:
pairs_df = pd.DataFrame(all_pairs, columns=['fPath1','fPath2','label','test_label'])

In [154]:
pairs_df.to_csv('smoke_pairs.csv')